In [ ]:
from Py2P.core import rec2p,sync
from matplotlib import pyplot as plt
import os
import shutil

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
datapath = 'suite2p\\plane0\\'
syncfile = 'rec_000_000.mat'
sequencefile = 'stim_dict.json'

trials_names = {0:"IPSI",1:"CONTRA",2:"BOTH"}

In [ ]:
synchro = sync.Sync()
synchro.generate_data_structure(syncfile,sequencefile,trials_names)
rec = rec2p.Rec2P(data_path=datapath, sync=synchro)
cells = rec.get_cells(keep_unresponsive=False)



In [ ]:
### analyze and plot populations ###
import numpy as np
from sklearn.decomposition import PCA

fingerprints = rec.compute_fingerprints(stim_trials_dict={'chirp':['BOTH']},normalize='z')

tsne_params =  {
            'n_components':2, 
            'verbose':1, 
            'metric':'cosine', 
            'early_exaggeration':10, 
            'perplexity':12, 
            'n_iter':4000, 
            'init':'pca', 
            'angle':0.9}

transformed = core.TSNE_embedding(fingerprints,**tsne_params)
# transformed = PCA(n_components=2).fit_transform(fingerprints)

labels = core.GMM(transformed,**{'n_components':2,'covariance_type':'full'})

pops = []
for n in np.unique(labels):

    indices = np.squeeze(np.argwhere(labels == n))
    c = []
    for i in indices:

        c.append(rec.get_responsive()[i])

    pops.append(c)

plot.plot_clusters(transformed,labels,algo='tsne',l1loc='lower right',save='TSNE_populations')
# plot.plot_clusters(transformed,labels,algo='pca',l1loc='lower right',save='PCA_populations')


In [ ]:
### plot ###

import numpy as np

ordered_trials = list(trials_names.values())
stim_names = sorted(synchro.stims_names)

for id,pop in enumerate(pops):

    save_path = "POPULATION_#%d"%id

    if os.path.isdir(save_path):

        shutil.rmtree(save_path)

    os.mkdir(save_path)

    pop_cells = [cells[c] for c in pop]


    ### plot heatmap

    fig, axs = plt.subplots(len(trials_names),len(stim_names),figsize=(20,20))

    fig.suptitle('POPULATION #%d, N:%d'%(id,len(pop_cells)), fontsize=16)

    for i,trial in enumerate(ordered_trials):

        for j,stim in enumerate(stim_names):

            if not isinstance(axs, np.ndarray):

                ax = axs

            elif len(stim_names)==1:

                ax=axs[i]

            elif len(ordered_trials)==1:

                ax=axs[j]

            else:
                ax=axs[i,j]


            if trial in synchro.sync_ds[stim]:


                plot.plot_averages_heatmap(pop_cells, 
                                            synchro, 
                                            stims=stim, 
                                            trials=trial, 
                                            type='dff', 
                                            stim_bar=False,
                                            vmin=None, 
                                            vmax=None, 
                                            normalize='z', 
                                            cb_label="\u0394F/F (z-score)",
                                            ax=ax)

            else:

                ax.axis("off")

            if i == 0: ax.set_title(stim,fontsize=15)

    if len(ordered_trials)>1:

        for ax, trial in zip(axs, ordered_trials):

            ax[0].set_ylabel(trial,fontsize=18)
    else:

        ax.set_ylabel(trial,fontsize=18)

    plt.savefig(r"%s/heatmaps_dff.png"%save_path, bbox_inches="tight")
    plt.close(fig)

    ###
    #plot full heatmap
    fig, axs = plt.subplots(1,figsize=(10,7))
    fig.suptitle('POPULATION #%d, N:%d'%(id,len(pop_cells)), fontsize=16)

    plot.plot_averages_heatmap(pop_cells,
                                synchro,
                                full="dff",
                                vmin=None, 
                                vmax=None, 
                                normalize='z', 
                                cb_label="\u0394F/F (z-score)", 
                                stim_bar=False,
                                ax=axs)
    
    plt.savefig(r"%s/full_heatmap_dff.png"%save_path, bbox_inches="tight")
    plt.close(fig)

    ### plot averaged responses

    plot.plot_multipleStim(pop_cells, 
                           synchro, 
                           average=True, 
                           save=True, 
                           save_path=save_path, 
                           stims=None, 
                           trials=None, 
                           full='dff',
                           share_x=False, 
                           share_y=False,
                           group_trials=True, 
                           legend=True)


    # ### plot all the cells

    # save_path_allcells = save_path+r"/all_cells"

    # if os.path.isdir(save_path_allcells):

    #     shutil.rmtree(save_path_allcells)

    # os.mkdir(save_path_allcells)

    # plot.plot_multipleStim(pop_cells, 
    #                        synchro, 
    #                        average=False, 
    #                        save=True, 
    #                        save_path=save_path_allcells, 
    #                        stims=None, 
    #                        trials=None, 
    #                        full='dff',
    #                        share_x=False, 
    #                        share_y=False,
    #                        group_trials=False, 
    #                        legend=True)
    



In [ ]:
## plot FOV
plot.plot_FOV(pops, rec, k=5)